In [1]:
#importing libraries 
%load_ext autotime
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from imblearn.over_sampling import SMOTE
from scipy import stats
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,f1_score,roc_auc_score,roc_curve

time: 1.12 s (started: 2022-05-18 16:03:00 +05:30)


In [2]:
#reading the data
train=pd.read_csv('train_file.csv')
test=pd.read_csv('test_file.csv')

time: 62.5 ms (started: 2022-05-18 16:03:01 +05:30)


In [3]:
#train data
train.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
0,25150,360000,2,2,2,25,-1,-1,-1,-1,...,3435,8870,2020,12590,3479,3446,8870,2020,27043,0
1,13470,240000,2,1,2,46,2,2,-2,-1,...,2240,1681,2267,0,0,2240,0,2267,3074,0
2,3092,320000,2,2,1,41,0,0,0,0,...,35409,35036,35939,5000,2000,2000,3000,3000,1500,0
3,13973,50000,2,2,2,24,-1,-1,-2,-2,...,0,0,0,0,0,0,0,0,0,1
4,10567,80000,1,3,1,52,-1,0,0,0,...,20561,21261,21457,1500,1500,1500,1028,876,1449,0


time: 132 ms (started: 2022-05-18 16:03:01 +05:30)


In [4]:
def modify(df):
    df['pay_avg']=df[['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']].mean(axis=1)
    df['BILL_AVG']=df[['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']].mean(axis=1)
    df['pay_amt_AVG']=df[['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']].mean(axis=1)
#     df.drop(['PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6','BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',],axis=1,inplace=True)
#     df.drop(['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'],axis=1,inplace=True)
    return df

time: 91.7 ms (started: 2022-05-18 16:03:01 +05:30)


In [5]:
modify(test)
modify(train)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month,pay_avg,BILL_AVG,pay_amt_AVG
0,25150,360000,2,2,2,25,-1,-1,-1,-1,...,12590,3479,3446,8870,2020,27043,0,-1.000000,8557.666667,9574.666667
1,13470,240000,2,1,2,46,2,2,-2,-1,...,0,0,2240,0,2267,3074,0,0.000000,1107.333333,1263.500000
2,3092,320000,2,2,1,41,0,0,0,0,...,5000,2000,2000,3000,3000,1500,0,0.000000,41459.666667,2750.000000
3,13973,50000,2,2,2,24,-1,-1,-2,-2,...,0,0,0,0,0,0,1,-1.666667,445.833333,0.000000
4,10567,80000,1,3,1,52,-1,0,0,0,...,1500,1500,1500,1028,876,1449,0,-0.166667,22841.000000,1308.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,6283,50000,2,2,2,31,1,-2,-2,-2,...,0,0,0,0,0,0,0,-1.500000,0.000000,0.000000
20996,25589,80000,2,2,1,37,0,0,0,0,...,5000,3000,2000,4000,2000,2000,0,0.000000,55198.500000,3000.000000
20997,4378,100000,1,1,1,52,1,2,2,2,...,0,3600,0,6000,3000,0,1,1.166667,25024.000000,2100.000000
20998,8604,90000,2,2,1,47,0,0,0,0,...,2000,2000,2000,2000,2000,2000,0,0.000000,28903.000000,2000.000000


time: 132 ms (started: 2022-05-18 16:03:01 +05:30)


In [6]:
def frequency_encoding(data,cat):
    
    """
    data: training data
    cat: categorical feature on which we have to apply frequency encoding techniques
 
    returns a feature will all categoris assigned to its frequency
    
    """

    
    
    freq_enc = data.groupby(cat).size()/len(data)
    return  data[cat].map(freq_enc)
 
#Applying frequency encoding to marriage feature
train['MARRIAGE'] = frequency_encoding(train,'MARRIAGE')
test['MARRIAGE'] = frequency_encoding(test,'MARRIAGE')

time: 37.2 ms (started: 2022-05-18 16:03:02 +05:30)


In [7]:
#label encoding on 'EDUCATION'
le = LabelEncoder()
train['EDUCATION'] = le.fit_transform(train['EDUCATION'])

test['EDUCATION'] = le.fit_transform(test['EDUCATION'])

time: 153 ms (started: 2022-05-18 16:03:02 +05:30)


In [8]:
#onehotencoding on 'SEX' 
enc = OneHotEncoder(handle_unknown='ignore')
train['SEX']=enc.fit_transform(train[['SEX']]).toarray()

test['SEX']=enc.fit_transform(test[['SEX']]).toarray()

time: 88.7 ms (started: 2022-05-18 16:03:02 +05:30)


In [9]:
#defining x and Y sets
Y=train['default_payment_next_month']
train.drop('default_payment_next_month',axis=1,inplace=True)
x=train

time: 58.9 ms (started: 2022-05-18 16:03:02 +05:30)


In [10]:
x.drop(['ID'],axis=1,inplace=True)

time: 66.9 ms (started: 2022-05-18 16:03:02 +05:30)


In [11]:
#applying smote for class balancing
sm = SMOTE(random_state=42)
x_res, Y_res= sm.fit_resample(x, Y)

time: 469 ms (started: 2022-05-18 16:03:02 +05:30)


In [12]:
#splitting the data into train and test set
X_train,X_test,y_train,y_test=train_test_split(x_res, Y_res,test_size=0.25,shuffle=True,random_state=42)

time: 15.3 ms (started: 2022-05-18 16:03:02 +05:30)


In [13]:
X_train.reset_index(drop=True, inplace=True)

time: 69.3 ms (started: 2022-05-18 16:03:02 +05:30)


In [14]:
def draw_bootstrap(X_train, y_train):
    
    """
    Params: X_train, y_train: training data to create subsets from
    
    returns: Bootstrapped train data and Out of bag data for testing
    
    """
    bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace = True))
    oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices]
    X_bootstrap = X_train.iloc[bootstrap_indices]
    y_bootstrap = y_train.iloc[bootstrap_indices].values.reshape(-1, 1)
    X_oob = X_train.iloc[oob_indices]
    y_oob = y_train.iloc[oob_indices]
    return X_bootstrap, y_bootstrap, X_oob, y_oob

time: 78.5 ms (started: 2022-05-18 16:03:03 +05:30)


In [15]:
def metrics(i,test,pred):
    """
    params: i: iterable, for ith number of dt model
            test data,prediction: to calculate metrics of the model
    
    """
    print("Metrics of {}th DT model".format(i))
    print("Accuracy:{}".format(accuracy_score(test, pred)))
    print("F1 Score: {}".format(f1_score(test, pred)))
    print("roc_auc Score: {}".format(roc_auc_score(test, pred)))
    

time: 98.8 ms (started: 2022-05-18 16:03:03 +05:30)


In [16]:
dts=[]
features={}
def random_forest_classifier(X_train, y_train):
    """ 
    
    Function to build a random forest from multiple decision tress trained on different subsets of train data
    
    """
    
    scores = []
    for i in range(0, 10):
        #generate 10 subsets of data
        X_bootstrap, y_bootstrap, X_oob, y_oob = draw_bootstrap(X_train, y_train)

        #column sampling
        features[i]=np.random.choice(X_train.columns,14,replace = False)
        #building the decision tree model
        dt=DecisionTreeClassifier(random_state=42,min_samples_split= 5, min_samples_leaf= 15, criterion='entropy')

        #fit train data
        dt.fit(X_bootstrap.loc[:,features[i]], y_bootstrap)
        dts.append(dt)
        y_pred=dt.predict(X_oob.loc[:,features[i]])
        # evaluate accuracy and store
        metrics(i,y_oob,y_pred)
        scores.append(accuracy_score(y_oob, y_pred))



    print("Average accuracy: {:.2f}%.".format(np.mean(scores) * 100))



time: 153 ms (started: 2022-05-18 16:03:03 +05:30)


In [17]:
#function call
random_forest_classifier(X_train, y_train)

Metrics of 0th DT model
Accuracy:0.7748388530784619
F1 Score: 0.7736312849162011
roc_auc Score: 0.7748806357136364
Metrics of 1th DT model
Accuracy:0.7931034482758621
F1 Score: 0.7903225806451613
roc_auc Score: 0.7931125918675076
Metrics of 2th DT model
Accuracy:0.8033969804618117
F1 Score: 0.7984981226533167
roc_auc Score: 0.8034868593489896
Metrics of 3th DT model
Accuracy:0.7187953151143335
F1 Score: 0.7174083622912228
roc_auc Score: 0.7187906564133784
Metrics of 4th DT model
Accuracy:0.7694941462337088
F1 Score: 0.7621652421652422
roc_auc Score: 0.7693103963178975
Metrics of 5th DT model
Accuracy:0.7807085310672112
F1 Score: 0.7735612535612536
roc_auc Score: 0.7805048972188692
Metrics of 6th DT model
Accuracy:0.7870216306156406
F1 Score: 0.7838324701643774
roc_auc Score: 0.787393968984802
Metrics of 7th DT model
Accuracy:0.7783825816485226
F1 Score: 0.7759685569904547
roc_auc Score: 0.7784343841800012
Metrics of 8th DT model
Accuracy:0.7369644636333444
F1 Score: 0.7346437346437347


In [18]:
#function to perform prediction of single instance using random forest
def predict_row(instance):
    
    
    """  
    params: single instance to perform prediction 
    
    prints: predicted class the instance belongs to
    
    """
    preds=[]
    
    
    for i in range(len(dts)):
                preds.append(dts[i].predict(instance[features[i]]))


    y_pred=stats.mode(np.array(preds), axis=None)[0][0]
    
   
    print("Prediction of given instance:{}".format(y_pred))

time: 565 µs (started: 2022-05-18 16:09:35 +05:30)


In [19]:
#function call
predict_row(X_test.iloc[:1])

Prediction of given instance:0
time: 132 ms (started: 2022-05-18 16:09:35 +05:30)


### Accuracy of the RF model

In [20]:
preds=[]
#y_pred=[]
for i in range(len(dts)):
        preds.append(dts[i].predict(X_test.loc[:,features[i]]))
y_pred=stats.mode(np.array(preds), axis=0)[0][0]

time: 313 ms (started: 2022-05-18 16:09:35 +05:30)


In [21]:
np.array(preds).shape

(10, 8178)

time: 2.39 ms (started: 2022-05-18 16:09:35 +05:30)


In [22]:
print("Accuracy of the random forest classifier:{}".format(accuracy_score(y_test, y_pred) * 100))

Accuracy of the random forest classifier:85.57104426510149
time: 106 ms (started: 2022-05-18 16:09:35 +05:30)
